In [1]:
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from ddpg_agent import Agent

In [60]:
class CartAcrobat:

    def __init__(self, l1=1, l2=1, m=(0.3, 0.2, 0.2), b=(0.2, 0.01, 0.01), g=9.81, rail_lims=(-3, 3), force_lims=(-20, 20), num_instances=1):
        self.g = np.float64(9.81)
        self.l1 = np.float64(l1)
        self.l2 = np.float64(l2)
        self.m = np.array(m, dtype=np.float64)
        self.b = np.array(b, dtype=np.float64)
        self.g = np.float64(g)
        self.rail_lims = np.array(rail_lims, dtype=np.float64)
        self.force_lims = np.array(force_lims, dtype=np.float64)

        # System dimensions
        self.n_u = 1  # inputs
        self.n_d = 3  # degrees of freedom
        self.n_q = 2 * self.n_d  # states

        self.init_state = [None]
        self.state = [None]
        self.time_elapsed = np.zeros(num_instances)
        self.num_of_instances = num_instances

    def update_state(self, q):
        # TODO: check if size of q coresponds to number of instances
        self.state = q
        self.init_state = q

    def get_position_to_plot(self):
        q = self.state[0]
        x = np.cumsum([q[0],
                       self.l1 * np.sin(q[1]),
                       self.l2 * np.sin(q[2])])
        y = np.cumsum([0,
                       self.l1 * np.cos(q[1]),
                       self.l2 * np.cos(q[2])])
        return (x, y)

    def energy(self):
        return 0.0
    
    def get_norm_hight(self):
        q = self.state
        h1 = np.cos(q[:, 1])
        h2 = (self.l1 * h1 + self.l2* np.cos(q[:, 2])) / (self.l1 + self.l2)
        return h1, h2

    def dstate_dt(self, u, dt=0.05):
        q = self.state
        theta1 = q[:, 1]
        theta2 = q[:, 2]
        dp = q[:, 3]
        dtheta1 = q[:, 4]
        dtheta2 = q[:, 5]

        b = [min(my_iter, 1 / dt - 1e-10) for my_iter in self.b]

        M = np.zeros((self.num_of_instances, 3,3))
        M[:, 0, 0] = np.sum(self.m)
        M[:, 0, 1] = self.l1 * (self.m[1] + self.m[2]) * np.cos(theta1)
        M[:, 0, 2] = self.m[2] * self.l2 * np.cos(theta2)

        M[:, 1, 0] = M[:, 0, 1]
        M[:, 1, 1] = (self.l1 ** 2) * (self.m[1] + self.m[2])
        M[:, 1, 2] = self.l1 * self.l2 * self.m[2] * np.cos(theta1 - theta2)

        M[:, 2, 0] = M[:, 0, 2]
        M[:, 2, 1] = M[:, 1, 2]
        M[:, 2, 2] = (self.l2 ** 2) * self.m[2]

        P1 = np.zeros((self.num_of_instances, 3,1))
        P1[:, 0, 0] = ((self.l1 * (self.m[1] + self.m[2]) * (dtheta1 ** 2) * np.sin(theta1))
                    + (self.m[2] * self.l2 * (dtheta2 ** 2) * np.sin(theta2)))
        P1[:, 1, 0] = ((-1) * self.l1 * self.l2 * self.m[2] * (dtheta2 ** 2) * np.sin(theta1 - theta2)
                    + self.g * (self.m[1] + self.m[2]) * self.l1 * np.sin(theta1))
        P1[:, 2, 0] = (self.l1 * self.l2 * self.m[2] * (dtheta1 ** 2) * np.sin(theta1 - theta2)
                    + self.g * self.l2 * self.m[2] * np.sin(theta2))

        P2 = b * self.state[:, self.n_d:]
        P2 = P2.reshape((5,3, 1))

        P3 = np.zeros((5, 3, 1))
        P3[:, 0, 0] = u.reshape(5)

        P = P1 - P2 + P3

        d_y = np.matmul(np.linalg.inv(M), P)
        return(d_y.reshape((5, 3)))


    def step(self, u, dt, disturb=0.0):
        done = False
        q = self.state
        q = np.array(q, dtype=np.float64)

        # Enforce input saturation and add disturbance
        u_act = np.clip(np.float64(u), self.force_lims[0], self.force_lims[1]) + disturb

        # Get numpy views of pose and twist, and compute accel
        pose = np.copy(q[:, :self.n_d])
        twist = np.copy(q[:, self.n_d:])
        accel = self.dstate_dt(u, dt=dt)

        # Partial-Verlet integrate state and enforce rail constraints
        pose_next = pose + dt * twist + 0.5 * (dt ** 2)
        done = np.logical_or(ca.state[:, 0] < ca.rail_lims[0], ca.state[:, 0] > ca.rail_lims[1] )
        pose_next[:, 0] = np.clip(pose_next[:, 0], ca.rail_lims[0], ca.rail_lims[1])
        
        twist_next = twist + dt * accel

        # Return next state = [pose, twist]
        pose_next[1:3] = np.mod(pose_next[1:3] + np.pi, 2 * np.pi) - np.pi  # unwrap angles onto [-pi, pi]
        self.update_state(np.concatenate([pose, twist], axis=1))
        self.time_elapsed += dt
        reward = self.get_norm_hight()
        reward += done * -1000
        if np.any(done):
            self.reset(hard=False, done=done)
        return self.state, reward, done, ''
    
    def reset(self, hard=True, done=None):
        if hard:
            self.state = self.init_state
        else:
            for i in done:
                if i == True:
                    self.state[i] = np.copy(self.init_state[i])
        return self.state

In [61]:
ca = CartAcrobat(num_instances=5)

In [67]:
Q = np.random.sample((ca.num_of_instances, ca.n_q))
U = np.zeros((ca.num_of_instances, ca.n_u))
ca.update_state(Q)

In [70]:
U

array([[0.],
       [0.],
       [0.],
       [3.],
       [0.]])

In [66]:
for i in range(10000):
    ca.step(U, 0.02)

array([[0.96469113, 0.43316453, 0.99436473, 0.93489094, 0.58738785,
        0.99878869],
       [0.90084831, 0.46424365, 0.03352145, 0.74443844, 0.71724308,
        0.87937846],
       [0.55203652, 0.60120756, 0.43551384, 0.65048701, 0.87102844,
        0.97938222],
       [0.59742915, 0.17261984, 0.62556372, 0.47572414, 0.90268639,
        0.85979531],
       [0.43586905, 0.19950139, 0.7198676 , 0.34785535, 0.88145084,
        0.41891073]])

In [53]:
twist

array([[0.93489094, 0.58738785, 0.99878869],
       [0.74443844, 0.71724308, 0.87937846],
       [0.65048701, 0.87102844, 0.97938222],
       [0.47572414, 0.90268639, 0.85979531],
       [0.34785535, 0.88145084, 0.41891073]])

In [48]:
q = ca.state
theta1 = q[:, 1]
theta2 = q[:, 2]
dp = q[:, 3]
dtheta1 = q[:, 4]
dtheta2 = q[:, 5]

In [51]:
(dtheta1 ** 2) * np.sin(theta1)

array([0.0007633 , 0.02330333, 0.30486201, 0.04081109, 0.00178623])

In [53]:
np.sin(theta1)

array([0.66540017, 0.46390301, 0.51931946, 0.37251185, 0.40049997])

In [56]:
b= [1,2,3]

In [60]:
b * ca.state[2:]

ValueError: operands could not be broadcast together with shapes (3,) (3,6) 

In [71]:
ca.state[:, :3]

array([[0.90390661, 0.72802976, 0.37270502],
       [0.48051261, 0.48239591, 0.77707817],
       [0.98829331, 0.54605441, 0.41513577],
       [0.06176076, 0.38171421, 0.7367006 ],
       [0.53826272, 0.41206242, 0.57471711]])

In [72]:
b * ca.state[:, :3]

array([[0.90390661, 1.45605951, 1.11811506],
       [0.48051261, 0.96479182, 2.33123452],
       [0.98829331, 1.09210882, 1.24540732],
       [0.06176076, 0.76342842, 2.21010179],
       [0.53826272, 0.82412485, 1.72415132]])